# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
file = 'City_Data.csv'
df = pd.read_csv(file)
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.65,115.33,57.00,51,0,20.09,AU,1592773346
1,mataura,-46.19,168.86,32.81,92,0,1.74,NZ,1592773347
2,hualmay,-11.10,-77.61,66.43,74,22,8.23,PE,1592773348
3,kavieng,-2.57,150.80,83.07,73,33,4.63,PG,1592773349
4,bredasdorp,-34.53,20.04,57.20,93,66,3.09,ZA,1592773350


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key=g_key)
coordinate = df[["Lat",'Lng']]
humidity = df['Humidity']
figu = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(coordinate, weights = humidity, dissipating=False, max_intensity=max(df["Humidity"]),point_radius=1) 
figu.add_layer(heatmap_layer)
figu

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
narrow = df.loc[(df['Max Temp']>70)&(df['Max Temp']<80)&(df['Wind Speed']<10)&(df['Cloudiness']==0),:]
narrow.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
68,herat,34.50,62.00,78.39,16,0,4.70,AF,1592773426
139,arraial do cabo,-22.97,-42.03,77.00,73,0,8.05,BR,1592773513
185,gamba,-2.65,10.00,71.94,93,0,5.01,GA,1592773567
235,pirapemas,-3.73,-44.22,77.65,73,0,3.18,BR,1592773626
265,villa florida,-26.38,-57.15,75.99,68,0,3.94,PY,1592773659


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df=narrow
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"keyword":"hotel","radius":5000,"key":g_key}
hotel_name=[]
for i in range(len(hotel_df['City'])):
    latitude=hotel_df.iloc[i,1]
    longitude=hotel_df.iloc[i,2]
    params["location"]=f"{latitude},{longitude}"
    try:
        hotel=requests.get(base_url,params=params).json()
        hotel_name.append(hotel["results"][0]["name"])
    except:
        hotel_name.append("No hotel found")
hotel_df["Hotel Name"]=hotel_name
hotel_df.head()

C:\Users\T-Mac\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
68,herat,34.50,62.00,78.39,16,0,4.70,AF,1592773426,No hotel found
139,arraial do cabo,-22.97,-42.03,77.00,73,0,8.05,BR,1592773513,Hotel da Canoa
185,gamba,-2.65,10.00,71.94,93,0,5.01,GA,1592773567,No hotel found
235,pirapemas,-3.73,-44.22,77.65,73,0,3.18,BR,1592773626,HOTEL DA LOURA
265,villa florida,-26.38,-57.15,75.99,68,0,3.94,PY,1592773659,Hotel Playa


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(locations,info_box_content=hotel_info)

# Display figure
figu.add_layer(markers)
figu.add_layer(hotel_layer)
figu


Figure(layout=FigureLayout(height='420px'))